In [2]:
from datetime import datetime, timezone
import pandas as pd
import requests
import json
import math
import ta
import yfinance as yf
from joblib import Parallel, delayed 

pd.set_option('display.max_columns', 100)


In [3]:

BINANCE_URL = 'https://api.binance.com/api/v3/klines'

NUM_CORES = 4

PROXIES = {
    'http': 'http://discproxy.virtual.uniandes.edu.co:443',
}

# Method to make API request to retrieve the last recorded price timestamp for a given asset ID



def round_minute(epoch_timestamp):
    # convert epoch timestamp to datetime object
    timestamp1 = datetime.utcfromtimestamp(epoch_timestamp)

    # round to nearest day
    rounded_timestamp1 = timestamp1.replace(
        second=0, microsecond=0, tzinfo=timezone.utc)

    # convert rounded timestamp back to epoch timestamp
    return int(rounded_timestamp1.timestamp())


def round_hour(epoch_timestamp):
    # convert epoch timestamp to datetime object
    timestamp2 = datetime.utcfromtimestamp(epoch_timestamp)

    # round to nearest hour
    rounded_timestamp2 = timestamp2.replace(
        minute=0, second=0, microsecond=0, tzinfo=timezone.utc)

    # convert rounded timestamp back to epoch timestamp
    return int(rounded_timestamp2.timestamp())


def round_day(epoch_timestamp):
    # convert epoch timestamp to datetime object
    timestamp3 = datetime.utcfromtimestamp(epoch_timestamp)

    # round to nearest day
    rounded_timestamp3 = timestamp3.replace(
        hour=0, minute=0, second=0, microsecond=0, tzinfo=timezone.utc)

    # convert rounded timestamp back to epoch timestamp
    return int(rounded_timestamp3.timestamp())


def get_data_from_api(
        symbol: str, interval: str, initial_timestamp: int, limit_timestamp: int):
    """ This function get the information from binance API
    """

    # Set working dates ------------------------------------------------------------------------------------
    # End of Looping Period Date
    # Load will go from initial_date to limit_date
    # Start date is inclusive

    # set fields
    fields = ['datetime', 'open', 'high', 'low', 'close', 'volume', 'close_time',
              'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol', 'ignore']

    counter = 0
    df_prices = pd.DataFrame(columns=fields)

    if interval == "minute":

        initial_timestamp = round_minute(initial_timestamp)
        limit_timestamp = round_minute(limit_timestamp)

        print("last time in the api: " +
              str(datetime.utcfromtimestamp(initial_timestamp)))
        print("Actual datetime: " + str(datetime.utcfromtimestamp(limit_timestamp)))

        if initial_timestamp + 60 < limit_timestamp:
            print("Si se trae data")
            initial_timestamp = initial_timestamp + 60
            limit_timestamp = limit_timestamp - 62
        else:
            print("Finalizo y no hay datos")
            return df_prices

        interval_binance = "1m"
        limit_binance = "960"
        following_period = 57600

        # We will work with time intervals of 16 hours when working with minutes to have 960 records per API call - LIMIT = 1000
        end_date_timestamp = initial_timestamp + following_period

    if interval == "hour":

        initial_timestamp = round_hour(initial_timestamp)
        limit_timestamp = round_hour(limit_timestamp)

        print("last time in the api: " +
              str(datetime.utcfromtimestamp(initial_timestamp)))
        print("Actual datetime: " + str(datetime.utcfromtimestamp(limit_timestamp)))

        if initial_timestamp + 3600 < limit_timestamp:
            print("Si se trae data")
            initial_timestamp = initial_timestamp + 3600
            limit_timestamp = limit_timestamp - 3602
        else:
            print("Finalizo y no hay datos")
            return df_prices

        interval_binance = "1h"
        limit_binance = "960"
        following_period = 144000

        # We will work with time intervals of 40 days when working with hours to have 960 records per API call - LIMIT = 1000
        end_date_timestamp = initial_timestamp + following_period

    if interval == "day":

        initial_timestamp = round_day(initial_timestamp)
        limit_timestamp = round_day(limit_timestamp)

        print("last time in the api: " +
              str(datetime.utcfromtimestamp(initial_timestamp)))
        print("Actual datetime: " + str(datetime.utcfromtimestamp(limit_timestamp)))

        if initial_timestamp + 86400 < limit_timestamp:
            print("Si se trae data")
            initial_timestamp = initial_timestamp + 86400
            limit_timestamp = limit_timestamp - 86402
        else:
            print("Finalizo y no hay datos")
            return df_prices

        interval_binance = "1d"
        limit_binance = "360"
        following_period = 31104000
        end_date_timestamp = initial_timestamp + following_period

    # Start time of function
    loop_start_time = datetime.now()

    # Loop through API calls ---------------------------------------------------------------------------
    while initial_timestamp < limit_timestamp:
        print("Entro loop")

        # Set dates to Binance API format
        start = str(initial_timestamp*1000)

        if end_date_timestamp > limit_timestamp:
            end = str(limit_timestamp*1000)
        else:
            end = str(end_date_timestamp*1000)

        par = {'symbol': symbol, 'interval': interval_binance,
               'startTime': start, 'endTime': end, 'limit': limit_binance}

        # API CALL
        response = requests.get(BINANCE_URL, params=par)
        json_data = json.loads(response.content)
        new_df = pd.DataFrame(json_data, columns=fields)
        df_prices = pd.concat([df_prices, new_df], axis=0)

        # Move to following period
        initial_timestamp = end_date_timestamp
        end_date_timestamp = initial_timestamp + following_period
        counter += 1

    # End time of function
    loop_end_time = datetime.now()

    time = loop_end_time - loop_start_time

    print('DONE', '\nDURATION:', time.days, 'DAYS', time.seconds//3600, 'HOURS',
          (time.seconds//60) % 60, 'MINUTES', time.seconds % 60, 'SECONDS.')
    print('API CALLS:', counter, ' ROWS:', len(df_prices))

    df_prices = df_prices.rename(columns={'datetime': 'unix_time',
                                          'open': 'open_price',
                                          'high': 'high_price',
                                          'low': 'low_price',
                                          'close': 'close_price'})

    df_prices['unix_time'] = df_prices['unix_time'].apply(
        lambda x: int(round(x / 1000)))
    
    # Apply the function to the 'unix_timestamp' column and create a new column called 'date'
    df_prices['timestamp_round_day'] = df_prices['unix_time'].apply(
        round_day)
    
    df_prices = df_prices.drop_duplicates()


    return df_prices



# Define a function to convert Unix timestamp to date string
# def unix_to_date(unix_timestamp):
#     datetime_obj = datetime.utcfromtimestamp(unix_timestamp)
#     date_str = datetime_obj.date().strftime('%Y-%m-%d')
#     return date_str


# method to add the thecnic indicators to the dataframe
def add_indicators(df):

    # momentum
    df['ao'] = ta.momentum.awesome_oscillator(high=df['high_price'], low=df['low_price'])
    df['kama'] = ta.momentum.kama(close=df['close_price'])
    df['ppo'] = ta.momentum.ppo(close=df['close_price'])
    df['pvo'] = ta.momentum.pvo(volume=df['volume'])
    df['roc'] = ta.momentum.roc(close=df['close_price'])
    df['rsi'] = ta.momentum.rsi(close=df['close_price'])
    df['stochrsi'] = ta.momentum.stochrsi(close=df['close_price'])
    df['stoch'] = ta.momentum.stoch(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['tsi'] = ta.momentum.tsi(close=df['close_price'])
    df['uo'] = ta.momentum.ultimate_oscillator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['wri'] = ta.momentum.williams_r(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # volume
    df['accdist'] = ta.volume.acc_dist_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['cmf'] = ta.volume.chaikin_money_flow(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['emv'] = ta.volume.ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['fi'] = ta.volume.force_index(
        close=df['close_price'], volume=df['volume'])
    df['mfi'] = ta.volume.money_flow_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['nvi'] = ta.volume.negative_volume_index(
        close=df['close_price'], volume=df['volume'])
    df['obv'] = ta.volume.on_balance_volume(
        close=df['close_price'], volume=df['volume'])
    df['smaemv'] = ta.volume.sma_ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['vpt'] = ta.volume.volume_price_trend(
        close=df['close_price'], volume=df['volume'])
    df['vwap'] = ta.volume.volume_weighted_average_price(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])

    # volatility
    df['atr'] = ta.volatility.average_true_range(
        high=df['high_price'], low=df['low_price'], close=df['close_price'])
    df['ulcer'] = ta.volatility.ulcer_index(close=df['close_price'])

    df['bbh'] = ta.volatility.bollinger_hband(close=df['close_price'])
    df['bbl'] = ta.volatility.bollinger_lband(close=df['close_price'])
    df['bbhi'] = ta.volatility.bollinger_hband_indicator(
        close=df['close_price'])
    df['bbli'] = ta.volatility.bollinger_lband_indicator(
        close=df['close_price'])
    df['bbmavg'] = ta.volatility.bollinger_mavg(close=df['close_price'])
    df['bb_pb'] = ta.volatility.bollinger_pband(close=df['close_price'])
    df['bb_wb'] = ta.volatility.bollinger_wband(close=df['close_price'])

    df['dchb'] = ta.volatility.donchian_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dclb'] = ta.volatility.donchian_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcmb'] = ta.volatility.donchian_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcpb'] = ta.volatility.donchian_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcwb'] = ta.volatility.donchian_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['kchb'] = ta.volatility.keltner_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclb'] = ta.volatility.keltner_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kchbi'] = ta.volatility.keltner_channel_hband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclbi'] = ta.volatility.keltner_channel_lband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcmb'] = ta.volatility.keltner_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcpb'] = ta.volatility.keltner_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcwb'] = ta.volatility.keltner_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # trend

    # En dos de estos tres hay algo que esta botando warning
    df['adx'] = ta.trend.adx(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['adx_neg'] = ta.trend.adx_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['adx_pos'] = ta.trend.adx_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['aroon_up'] = ta.trend.aroon_up(close=df['close_price'])
    df['aroon_down'] = ta.trend.aroon_down(close=df['close_price'])

    df['cci'] = ta.trend.cci(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['dpo'] = ta.trend.dpo(close=df['close_price'])
    df['kst'] = ta.trend.kst(close=df['close_price'])
    df['kst_sig'] = ta.trend.kst_sig(close=df['close_price'])
    df['ema'] = ta.trend.ema_indicator(close=df['close_price'])

    df['ichimoku_a'] = ta.trend.ichimoku_a(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_b'] = ta.trend.ichimoku_b(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_base_line'] = ta.trend.ichimoku_base_line(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_conversion_line'] = ta.trend.ichimoku_conversion_line(
        high=df['high_price'], low=df['low_price'])

    df['macd'] = ta.trend.macd(close=df['close_price'])
    df['macd_diff'] = ta.trend.macd_diff(close=df['close_price'])
    df['macd_signal'] = ta.trend.macd_signal(close=df['close_price'])

    df['mi'] = ta.trend.mass_index(high=df['high_price'], low=df['low_price'])
    df['sma'] = ta.trend.sma_indicator(close=df['close_price'])
    df['wma'] = ta.trend.wma_indicator(close=df['close_price'])
    df['stc'] = ta.trend.stc(close=df['close_price'])
    df['trix'] = ta.trend.trix(close=df['close_price'])

    df['vi_pos'] = ta.trend.vortex_indicator_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['vi_neg'] = ta.trend.vortex_indicator_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    return df


def get_extra_assets_data(start_date, end_date, name, ticker):

    print("Getting data for: " + name + " "+ticker)

    df = yf.Ticker(ticker).history(
        interval="1d", start=start_date, end=end_date)
    df = df.reset_index(drop=False)


    df['unix_time'] = df['Date'].dt.date.apply(
        lambda x: int(pd.Timestamp(x).timestamp()))

    # Apply the function to the 'unix_timestamp' column and create a new column called 'date'
    df['timestamp_round_day'] = df['unix_time'].apply(round_day)

    df.drop(['Open', 'High', 'Low', 'Volume', 'Dividends',
            'Stock Splits', 'Date', 'unix_time'], axis=1, inplace=True)

    df.rename(columns={'Close': name}, inplace=True)

    print("DONE getting data for" + name + " "+ticker)
    return df


def create_df_extra_assets_data(start_date, end_date):

    dict_assets_extra = {
        "gold_price":  "GC=F",
        "silver_price":  "SI=F",
        "natural_gas_price":  "NG=F",
        "cotton_price":  "CT=F",
        "coffee_price":  "KC=F",
        "sugar_price":  "SB=F",
        "cocoa_price":  "CC=F",
        "rice_price":  "ZR=F",
        "corn_price":  "ZC=F",
        "wheat_price":  "KE=F",
        "soybean_price":  "ZS=F",
        "oats_price":  "ZO=F",
        "spy500_price":  "ES=F",
        "dow_jones_price":  "YM=F",
        "nasdaq_price":  "NQ=F",
        "russell_2000_price":  "RTY=F",
        "us_10_year_treasury_price":  "ZN=F",
        "us_5_year_treasury_price":  "ZF=F",
        "us_2_year_treasury_price":  "ZT=F",
        "usbond_price":  "ZB=F"
    }

    df_extra_assets_data = pd.DataFrame()
    count = 0

    for key, value in dict_assets_extra.items():

        if count == 0:
            df_extra_assets_data = get_extra_assets_data(
                start_date, end_date, name=key, ticker=value)

        else:
            df_extra_assets_data = pd.merge(df_extra_assets_data, get_extra_assets_data(
                start_date, end_date, name=key, ticker=value), on='timestamp_round_day', how='outer')

        count += 1

    return df_extra_assets_data


In [12]:
# Si se quiere tener la data de los indicadores se debe insetar información de 100 eventos en el pasado

initial_timestamp = 1653103875
final_timestamp = int(datetime.now().timestamp())

# Ingrese el symbolo de la cryptomoneda
symbol = "ADAUSDT"


# el intervalo puede ser solo "day", "hour", "minute"
interval = "minute"


# Get the prices from the BINANCE API
df_prices_final = get_data_from_api(symbol=symbol, interval=interval, initial_timestamp=initial_timestamp, limit_timestamp=final_timestamp)


df_prices_final['open_price'] = df_prices_final['open_price'].astype(float)
df_prices_final['high_price'] = df_prices_final['high_price'].astype(float)
df_prices_final['low_price'] = df_prices_final['low_price'].astype(float)
df_prices_final['close_price'] = df_prices_final['close_price'].astype(float)
df_prices_final['volume'] = df_prices_final['volume'].astype(float)

# df_prices_wit_indicators = add_indicators(df_prices_final)

# df_extra_assets_data = create_df_extra_assets_data(start_date=initial_timestamp, end_date=final_timestamp)

# df_final = pd.merge(df_prices_wit_indicators, df_extra_assets_data , on='timestamp_round_day', how='left')

# df_final.drop(['timestamp_round_day'], axis=1, inplace=True)



last time in the api: 2022-05-21 03:31:00
Actual datetime: 2023-04-26 03:47:00
Si se trae data
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Entro loop
Ent

In [14]:
df = df_prices_final.drop_duplicates()

# check for duplicate values in the 'Name' column
duplicates = df['unix_time'].duplicated(keep=False)

# print the duplicate values
df[duplicates]


,unix_time,open_price,high_price,low_price,close_price,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore,timestamp_round_day


In [13]:

# check for duplicate values in the 'Name' column
duplicates = df_prices_final['unix_time'].duplicated(keep=False)

# print the duplicate values
df_prices_final[duplicates]

,unix_time,open_price,high_price,low_price,close_price,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore,timestamp_round_day
880,1679715120,0.3598,0.3603,0.3598,0.3602,33225.1,1679715179999,11966.94187000,49,22243.20000000,8010.63305000,0,1679702400
0,1679715120,0.3598,0.3603,0.3598,0.3602,33225.1,1679715179999,11966.94187000,49,22243.20000000,8010.63305000,0,1679702400
